In [1]:
import os
import requests
import json
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from cartoframes.viz import Map, Layer
from pymongo import MongoClient
from pymongo import GEOSPHERE
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
from folium import CircleMarker
load_dotenv()

True

In [2]:
client = MongoClient("localhost:27017")
db = client["Ironhack"] # I or i 
client_id= os.getenv("CLIENT_ID")
client_s= os.getenv("CLIENT_SECRET")
client_key=os.getenv("API_KEY")

In [3]:
# At first the choice was made to focus on Santa Monica since this is where GTA takes places and hosts host of gamining companies
initial_lat = 34.0218593
initial_long = -118.498265

In [4]:
LA_map = folium.Map(location = [initial_lat, initial_long], zoom_start=12.5)
LA_map

In [5]:
companies = db.get_collection('companies') # C or c

### function for creating the dataframe of all the info needed

In [6]:
# function for creating the dataframe of all the info needed

# lets drop some more, one of the most important requiremetns is having design companies nearby.
# We should remove all comnaies that are not design companies 

def Company_in_cali(c):
    filter_ = {'$and':[{'offices': {'$exists': 1}},{'offices.latitude':{'$type':1}},{'offices.longitude':{'$type': 1}}]}
    projection = {"_id":0,'name':1,'founded_year':1,'offices.country_code':1,"offices.state_code":1,'offices.city':1,'offices.address1':1,'offices.address2':1,'offices.latitude':1,'offices.longitude':1}
    sorting = {'offices.country_code':-1}
    lis = list(companies.find(filter_, projection).sort("offices.country_code", -1))
    df = pd.DataFrame(lis).explode("offices").reset_index(drop=True)
    df = pd.concat([df, df["offices"].apply(pd.Series)], axis=1).reset_index(drop=True)
    df.dropna(subset=["latitude"],inplace=True)
    df.dropna(subset=["founded_year"],inplace=True)
    df.dropna(subset=["address1"],inplace=True)
    df.dropna(subset=["address2"],inplace=True)
    df.dropna(subset=["city"],inplace=True)
    newcols = [element for element in df.columns if element != 'offices']
    df=df[newcols].reset_index(drop=True)
    df=df[df["country_code"]=="USA"]
    df=df[df['state_code'] == 'CA'].reset_index(drop=True)
    df = df.drop(columns="founded_year")
    return df


In [7]:
df= Company_in_cali(companies)
df

,name,address1,address2,city,state_code,country_code,latitude,longitude
0,Yola,201 Mission Street,Suite 2250,San Francisco,CA,USA,37.791115,-122.395475
1,Buongiorno,485 Alberto Way,Suite 210,Los Gatos,CA,USA,37.229898,-121.971853
2,InMobi,475 Brannan,Suite 420,San Francisco,CA,USA,37.446550,-122.170921
3,LinQpay,,,Carlsbad,CA,USA,33.095680,-117.244034
4,Sparkpr,2 Bryant Street,Suite 100,San Francisco,CA,USA,37.787394,-122.388442
...,...,...,...,...,...,...,...,...
2396,Cantimer,3700 Haven Court,,Menlo Park,CA,USA,37.484616,-122.181753
2397,Celestial Semiconductor,"2635 North 1st Street, Suite 150",,San Jose,CA,USA,37.386417,-121.929170
2398,Purfresh,"1350 Willow Road,",Suite 102,Menlo Park,CA,USA,37.477008,-122.151599
2399,MyChances,,,Menlo Park,CA,USA,37.448491,-122.180281


## Dataframe of all the employees and their count and salaries

- 20 Designers
- 5 UI/UX Engineers
- 10 Frontend Developers
- 15 Data Engineers
- 5 Backend Developers
- 20 Account Managers
- 1 Maintenance guy that loves basketball
- 10 Executives
- 1 CEO/President.

In [8]:
# Dataframe of all the employees and their count and salaries

employee_table = {'Title': ['Game designer', 'UX/UI', 'Front-end developer', 'Data engineer', 'Back-end developer',
                      'Account manager', 'Maintenance guy', 'Executive', 'CEO' ],
                'Salary': [115568, 104266, 87240, 112493, 80258, 58670, 40482, 52522, 148598],
                'Count': [20, 5, 10, 15, 5, 20, 1, 10,1]
                 } 
# We can sue this to weigh our decision, Dani did something like this,
# I think its good for reasoning why something is more important than another
# salaries are from glassdoor and are average salaries 

df_employee_table = pd.DataFrame(employee_table).explode("Title")
df_employee_table['total spending'] = df_employee_table.eval('Salary*Count')
df_employee_table.sort_values(by=['total spending'], ascending=False)

,Title,Salary,Count,total spending
0,Game designer,115568,20,2311360
3,Data engineer,112493,15,1687395
5,Account manager,58670,20,1173400
2,Front-end developer,87240,10,872400
7,Executive,52522,10,525220
1,UX/UI,104266,5,521330
4,Back-end developer,80258,5,401290
8,CEO,148598,1,148598
6,Maintenance guy,40482,1,40482


## Heatmapping function

In [9]:

def Heat_map_cali(df,radius):
    m = folium.Map(location = [34.0218593,-118.498265], zoom_start=10)
    HeatMap(data=df[["latitude","longitude"]],radius=radius).add_to(m)
    return m


In [10]:
Heat_map_cali(df,15)

## Foursquare api Work soo far


In [11]:
query = {"offices.city": "Los Angeles"}
projection = {"_id":0,"name":1,"offices.city":1, "country_code":1,
              "offices.latitude":1, "offices.longitude":1,"office.city":1}

list(companies.find(query, projection))

[{'name': 'Helio',
  'offices': [{'city': 'Los Angeles',
    'latitude': 34.057498,
    'longitude': -118.446596}]},
 {'name': 'Stickam',
  'offices': [{'city': 'Los Angeles',
    'latitude': 34.051409,
    'longitude': -118.254558}]},
 {'name': 'Ustream',
  'offices': [{'city': 'San Francisco',
    'latitude': 37.392936,
    'longitude': -122.07948},
   {'city': 'Los Angeles', 'latitude': None, 'longitude': None},
   {'city': 'Budapest', 'latitude': None, 'longitude': None}]},
 {'name': 'TripUp',
  'offices': [{'city': 'Los Angeles',
    'latitude': 34.052187,
    'longitude': -118.243425}]},
 {'name': 'Eventful',
  'offices': [{'city': 'San Diego',
    'latitude': 32.872081,
    'longitude': -117.206292},
   {'city': 'Los Angeles', 'latitude': None, 'longitude': None}]},
 {'name': 'ScanScout',
  'offices': [{'city': 'Boston',
    'latitude': 42.340889,
    'longitude': -71.051931},
   {'city': 'New York', 'latitude': 40.739833, 'longitude': -73.993158},
   {'city': 'Los Angeles', 'la

In [12]:
len(list(companies.find(query, projection)))

219

In [13]:
LosAngeles = list(companies.find(query, projection))

In [14]:
LosAngeles

[{'name': 'Helio',
  'offices': [{'city': 'Los Angeles',
    'latitude': 34.057498,
    'longitude': -118.446596}]},
 {'name': 'Stickam',
  'offices': [{'city': 'Los Angeles',
    'latitude': 34.051409,
    'longitude': -118.254558}]},
 {'name': 'Ustream',
  'offices': [{'city': 'San Francisco',
    'latitude': 37.392936,
    'longitude': -122.07948},
   {'city': 'Los Angeles', 'latitude': None, 'longitude': None},
   {'city': 'Budapest', 'latitude': None, 'longitude': None}]},
 {'name': 'TripUp',
  'offices': [{'city': 'Los Angeles',
    'latitude': 34.052187,
    'longitude': -118.243425}]},
 {'name': 'Eventful',
  'offices': [{'city': 'San Diego',
    'latitude': 32.872081,
    'longitude': -117.206292},
   {'city': 'Los Angeles', 'latitude': None, 'longitude': None}]},
 {'name': 'ScanScout',
  'offices': [{'city': 'Boston',
    'latitude': 42.340889,
    'longitude': -71.051931},
   {'city': 'New York', 'latitude': 40.739833, 'longitude': -73.993158},
   {'city': 'Los Angeles', 'la

In [15]:
LosAngeles_new = []
for i in LosAngeles:
    for j in i['offices']:
        if j['city'] == 'Los Angeles':
            city = j['city']
            lat = j['latitude']
            lon = j['longitude']
            name = i['name']
            type_ = ({"typepoint": 
                      {"type": "Point", 
                         "coordinates": [j['latitude'], j['longitude']]}})
            
            LosAngeles_new.append({"name":name, "lat":lat, "long":lon, "type":type_})
LosAngeles_new

[{'name': 'Helio',
  'lat': 34.057498,
  'long': -118.446596,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [34.057498, -118.446596]}}},
 {'name': 'Stickam',
  'lat': 34.051409,
  'long': -118.254558,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [34.051409, -118.254558]}}},
 {'name': 'Ustream',
  'lat': None,
  'long': None,
  'type': {'typepoint': {'type': 'Point', 'coordinates': [None, None]}}},
 {'name': 'TripUp',
  'lat': 34.052187,
  'long': -118.243425,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [34.052187, -118.243425]}}},
 {'name': 'Eventful',
  'lat': None,
  'long': None,
  'type': {'typepoint': {'type': 'Point', 'coordinates': [None, None]}}},
 {'name': 'ScanScout',
  'lat': 34.050887,
  'long': -118.459995,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [34.050887, -118.459995]}}},
 {'name': 'WebbAlert',
  'lat': 34.052187,
  'long': -118.243425,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [34.0

In [16]:
len(LosAngeles_new)

220

In [17]:
df_LosAngeles = pd.DataFrame(LosAngeles_new)
df_LosAngeles

,name,lat,long,type
0,Helio,34.057498,-118.446596,"{'typepoint': {'type': 'Point', 'coordinates':..."
1,Stickam,34.051409,-118.254558,"{'typepoint': {'type': 'Point', 'coordinates':..."
2,Ustream,NaN,NaN,"{'typepoint': {'type': 'Point', 'coordinates':..."
3,TripUp,34.052187,-118.243425,"{'typepoint': {'type': 'Point', 'coordinates':..."
4,Eventful,NaN,NaN,"{'typepoint': {'type': 'Point', 'coordinates':..."
...,...,...,...,...
215,California 7 Studios,NaN,NaN,"{'typepoint': {'type': 'Point', 'coordinates':..."
216,Pipit Interactive,34.045499,-118.470360,"{'typepoint': {'type': 'Point', 'coordinates':..."
217,Top Rated Website Designers,33.993326,-118.398784,"{'typepoint': {'type': 'Point', 'coordinates':..."
218,SOA Software,NaN,NaN,"{'typepoint': {'type': 'Point', 'coordinates':..."


In [18]:
df_LosAngeles.isna().sum()

name     0
lat     52
long    52
type     0
dtype: int64

In [19]:
df_LosAngeles_copy = df_LosAngeles.copy()

In [20]:
df_LosAngeles_copy.dropna(subset=['lat', 'long'],inplace=True)

In [21]:
df_LosAngeles_copy.isna().sum()


name    0
lat     0
long    0
type    0
dtype: int64

In [22]:
len(df_LosAngeles_copy)

168

In [23]:
df_LosAngeles_copy

,name,lat,long,type
0,Helio,34.057498,-118.446596,"{'typepoint': {'type': 'Point', 'coordinates':..."
1,Stickam,34.051409,-118.254558,"{'typepoint': {'type': 'Point', 'coordinates':..."
3,TripUp,34.052187,-118.243425,"{'typepoint': {'type': 'Point', 'coordinates':..."
5,ScanScout,34.050887,-118.459995,"{'typepoint': {'type': 'Point', 'coordinates':..."
6,WebbAlert,34.052187,-118.243425,"{'typepoint': {'type': 'Point', 'coordinates':..."
...,...,...,...,...
210,AECOM Technology,34.050796,-118.256861,"{'typepoint': {'type': 'Point', 'coordinates':..."
214,Culture Jam,34.013715,-118.466060,"{'typepoint': {'type': 'Point', 'coordinates':..."
216,Pipit Interactive,34.045499,-118.470360,"{'typepoint': {'type': 'Point', 'coordinates':..."
217,Top Rated Website Designers,33.993326,-118.398784,"{'typepoint': {'type': 'Point', 'coordinates':..."


In [24]:
LA_group = folium.FeatureGroup(name="Los Angeles")

In [25]:
HeatMap(data=df_LosAngeles_copy[["lat", "long"]], radius=15,
       gradient = {0.5: 'blue', 0.5:'green', 0.65: 'yellow', 1: 'red'}).add_to(LA_group)

In [26]:
LA_group.add_to(LA_map)


In [27]:
LA_map

In [28]:
SM_lat = 34.0195
SM_long = -118.4912

In [29]:
SM_map = folium.Map(location = [SM_lat, SM_long], zoom_start=10)
folium.Marker([SM_lat, SM_long]).add_to(SM_map)
folium.Circle([SM_lat, SM_long], radius=500).add_to(SM_map)
folium.Circle([SM_lat, SM_long], radius=1000).add_to(SM_map)
folium.Circle([SM_lat, SM_long], radius=3000).add_to(SM_map)

In [30]:
SM_map

In [31]:
SM_lat_updated = 34.03713832766421
SM_long_updated = -118.46959297037199

In [32]:
SM_map_updated = folium.Map(location = [SM_lat_updated, SM_long_updated], zoom_start=10)
folium.Marker([SM_lat_updated, SM_long_updated]).add_to(SM_map)
folium.Circle([SM_lat_updated, SM_long_updated], radius=500, color="red").add_to(SM_map_updated)
folium.Circle([SM_lat_updated, SM_long_updated], radius=1000, color="yellow").add_to(SM_map_updated)
folium.Circle([SM_lat_updated, SM_long_updated], radius=2000, color="green").add_to(SM_map_updated)
folium.Circle([SM_lat_updated, SM_long_updated], radius=3000, color="blue").add_to(SM_map_updated)

In [33]:
SM_map_updated

In [34]:
def querySM(query, location, radius):

    if " " in query.strip():
           query_ok = query.strip().replace(" ","%20")
    else:
           query_ok = query
    url = f"https://api.foursquare.com/v3/places/search?query={query_ok}&ll={str(location[0]).strip()}%2C{str(location[1]).strip()}&radius={radius}&limit=50"
    headers =  {"Accept": "application/json","Authorization":  f"{client_key}"
    }
    response = requests.get(url, headers=headers)
    
    if response:
        response=response.json()['results']
        places_list = [ {"name" : i["name"],
                      "lat"  : i["geocodes"]["main"]["latitude"],
                      "long" : i["geocodes"]["main"]["longitude"],
                      "type" : {"typepoint":
                                  {"type": "Point",
                                   "coordinates": [lat, lon]}}} for i in response]
        return places_list

In [35]:
def queryAirport(query, location, radius):

    if " " in query.strip():
           query_ok = query.strip().replace(" ","%20")
    else:
           query_ok = query
    url = f"https://api.foursquare.com/v3/places/search?query={query_ok}&ll={str(location[0]).strip()}%2C{str(location[1]).strip()}&radius={radius}&categories=19040&limit=50"
    headers =  {"Accept": "application/json","Authorization":  f"{client_key}"
    }
    response = requests.get(url, headers=headers)
    
    if response:
        response=response.json()['results']
        places_list = [ {"name" : i["name"],
                      "lat"  : i["geocodes"]["main"]["latitude"],
                      "long" : i["geocodes"]["main"]["longitude"],
                      "type" : {"typepoint":
                                  {"type": "Point",
                                   "coordinates": [lat, lon]}}} for i in response]
        return places_list
    


In [36]:
def queryStadium(query, location, radius):

    if " " in query.strip():
           query_ok = query.strip().replace(" ","%20")
    else:
           query_ok = query
    url = f"https://api.foursquare.com/v3/places/search?query={query_ok}&ll={str(location[0]).strip()}%2C{str(location[1]).strip()}&radius={radius}&limit=50"
    headers =  {"Accept": "application/json","Authorization":  f"{client_key}"
    }
    response = requests.get(url, headers=headers)
    
    if response:
        response=response.json()['results']
        places_list = [ {"name" : i["name"],
                      "lat"  : i["geocodes"]["main"]["latitude"],
                      "long" : i["geocodes"]["main"]["longitude"],
                      "type" : {"typepoint":
                                  {"type": "Point",
                                   "coordinates": [lat, lon]}}} for i in response]
        return places_list

In [37]:
# Why is noting being save into the dataframe?
 # we have tried differend radius, Something clearly happens here as this is where loading gets slow


In [38]:
daycare = querySM("daycare",[34.0522, -118.2437],50000)

In [39]:
design_studio = querySM("Design",[34.0522, -118.2437],50000)

In [40]:
airport = queryAirport("LAX",[34.0522, -118.2437],50000)

In [41]:
bar = querySM("bar",[34.0522, -118.2437],50000)

In [42]:
#starbucks = querySM("starbucks",[34.0522, -118.2437],50000)


In [43]:
basketball_stadium = queryStadium("Pauley Pavilion",[34.0522, -118.2437],50000)

In [44]:
pet_haircut = querySM("Pet grooming",[34.0522, -118.2437],50000)

In [45]:
vegan = querySM("vegan_vegetarian",[34.0522, -118.2437],50000)

In [46]:
df_airport = pd.DataFrame(data=airport)
df_bar = pd.DataFrame(data=bar)
#df_starbucks = pd.DataFrame(data=starbucks)
df_daycare = pd.DataFrame(data=daycare)
df_pet = pd.DataFrame(data=pet_haircut)
df_basketball_stadium = pd.DataFrame(data=basketball_stadium)
df_vegan = pd.DataFrame(data=vegan)
df_design = pd.DataFrame(data=design_studio)

In [47]:
from folium.plugins import MarkerCluster

In [48]:
#add airport

In [49]:
daycare_group = folium.FeatureGroup(name="daycare")

for index, row in df_daycare.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="red",
            prefix="fa",
            icon="baby-carriage",
            icon_color = "white")
    
    daycare_group.add_child(Marker(**venue, icon=icon))
    SM_map_updated.add_child(daycare_group)

In [50]:
bar_group = folium.FeatureGroup(name="Bar")

for index, row in df_bar.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="yellow",
            prefix="fa",
            icon="beer",
            icon_color = "white")
    
    bar_group.add_child(Marker(**venue, icon=icon))
    SM_map_updated.add_child(bar_group)

/var/folders/cm/fc2_w_693_s292_lj65rkvnw0000gn/T/ipykernel_5041/960135678.py:7: UserWarning: color argument of Icon should be one of: {'purple', 'lightgreen', 'red', 'beige', 'green', 'darkblue', 'blue', 'darkgreen', 'cadetblue', 'gray', 'lightred', 'pink', 'black', 'darkpurple', 'lightgray', 'lightblue', 'white', 'darkred', 'orange'}.
  icon = Icon (


In [51]:
design_group = folium.FeatureGroup(name="Design Studio")

for index, row in df_design.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="green",
            prefix="fa",
            icon="desktop",
            icon_color = "white")
    
    design_group.add_child(Marker(**venue, icon=icon))
    SM_map_updated.add_child(design_group)

In [52]:
airport_group = folium.FeatureGroup(name="Airport")

for index, row in df_airport.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="purple",
            prefix="fa",
            icon="plane",
            icon_color = "white")
    
    airport_group.add_child(Marker(**venue, icon=icon))
    SM_map_updated.add_child(airport_group)

In [53]:
"""starbucks_group = folium.FeatureGroup(name="Starbucks")

for index, row in df_starbucks.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="darkred",
            prefix="fa",
            icon="coffee",
            icon_color = "white")
    
    starbucks_group.add_child(Marker(**venue, icon=icon))
    SM_map_updated.add_child(starbucks_group)"""

'starbucks_group = folium.FeatureGroup(name="Starbucks")\n\nfor index, row in df_starbucks.iterrows():\n    \n    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}\n    \n    icon = Icon (\n            color="darkred",\n            prefix="fa",\n            icon="coffee",\n            icon_color = "white")\n    \n    starbucks_group.add_child(Marker(**venue, icon=icon))\n    SM_map_updated.add_child(starbucks_group)'

In [54]:
vegan_group = folium.FeatureGroup(name="Vegan Food")

for index, row in df_vegan.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="beige",
            prefix="fa",
            icon="cannabis",
            icon_color = "white")
    
    vegan_group.add_child(Marker(**venue, icon=icon))
    SM_map_updated.add_child(vegan_group)

In [55]:
basket_group = folium.FeatureGroup(name="Basket")

for index, row in df_basketball_stadium.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="orange",
            prefix="fa",
            icon="fa-dribbble",
            icon_color = "white")
    
    basket_group.add_child(Marker(**venue, icon=icon))
    SM_map_updated.add_child(basket_group)


In [56]:
pet_group = folium.FeatureGroup(name="Pet")

for index, row in df_pet.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="lightgray",
            prefix="fa",
            icon="paw",
            icon_color = "white")
    
    pet_group.add_child(Marker(**venue, icon=icon))
    SM_map_updated.add_child(pet_group)


In [57]:
offices_group = folium.FeatureGroup(name="offices")

for index, row in df_LosAngeles_copy.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="red",
            prefix="fa",
            icon="buildings",
            icon_color = "white")
    
    offices_group.add_child(Marker(**venue, icon=icon))
    SM_map_updated.add_child(offices_group)

In [58]:
folium.LayerControl(collapsed=False).add_to(SM_map_updated)

In [59]:
SM_map_updated

In [60]:
df_design

,name,lat,long,type
0,Dreamworks Animation,34.157486,-118.285447,"{'typepoint': {'type': 'Point', 'coordinates':..."
1,Silver Screening Room - Pacific Design Center,34.084190,-118.381896,"{'typepoint': {'type': 'Point', 'coordinates':..."
2,Pacific Design Center,34.082316,-118.382371,"{'typepoint': {'type': 'Point', 'coordinates':..."
3,Garrett Leight California Optical,33.989860,-118.464003,"{'typepoint': {'type': 'Point', 'coordinates':..."
4,Rolling Greens Nursery,34.076393,-118.353608,"{'typepoint': {'type': 'Point', 'coordinates':..."
5,Heath Ceramics,34.076347,-118.353961,"{'typepoint': {'type': 'Point', 'coordinates':..."
6,Rancho Tailors & Cleaners,34.041090,-118.423980,"{'typepoint': {'type': 'Point', 'coordinates':..."
7,South Coast Collection,33.694367,-117.925454,"{'typepoint': {'type': 'Point', 'coordinates':..."
8,Blueprint Furniture,34.053671,-118.378767,"{'typepoint': {'type': 'Point', 'coordinates':..."
9,Cleveland Art,34.048637,-118.232390,"{'typepoint': {'type': 'Point', 'coordinates':..."
